<a href="https://colab.research.google.com/github/theMistletoe/she-copilot/blob/main/she_copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ライブラリのインストール
!pip install openai
!pip install langchain
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

#@markdown **OPENAI_API_KEYの設定**
OPENAI_API_KEY = "" #@param {type:"string"}
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# ライブラリ定義
import librosa
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
from datetime import datetime

SAMPLE_RATE = 16000

In [ ]:
# 録音処理
# 参考リンク: https://colab.research.google.com/github/espnet/notebook/blob/master/espnet2_asr_realtime_demo.ipynb#scrollTo=8ICdjXlSidsS
# https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, file_name='audio.wav', sample_rate=16000):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(file_name, 'wb+') as f:
    f.write(b)
  return librosa.load(file_name, sr=sample_rate)

In [ ]:
# 録音&ファイル出力

output_audio_file = "audio_%s.wav"%datetime.now().strftime('%Y%m%d%H%M%S')

#@markdown **話す時間(秒)の設定**
#@markdown 考えていることを話す時間を設定してください。

import warnings
warnings.filterwarnings('ignore')
seconds = 60 #@param {type:"integer"}
print(f"あなたの今悩んでいることについて、マイクに向けて%d秒お話ししてください..."%seconds, end="")
speech, sample_rate = record(seconds, file_name=output_audio_file, sample_rate=SAMPLE_RATE)
print("完成。")



マイクに向けて180秒お話ししてください...

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored

In [ ]:
# テキスト化処理

import openai
# from pydub import AudioSegment

# print(output_audio_file)

# 長尺用
# song = AudioSegment.from_wav(output_audio_file)
# ten_minutes = 10 * 60 * 1000
# first_10_minutes = song[:ten_minutes]
# first_10_minutes.export("audio_10.wav", format="mp3")

audio_file= open("audio_20230623165756.wav", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)
print(transcript["text"])

talk_script = transcript["text"]

今、GPTを使って自分の考え、思考を整理するためのツールを作っています。 人間は考える際に、脳の中の煩雑な情報を一度構造化、整理することで、より深い理解や洞察を得られると考えています。 その考えを持って、考えを整理し構造化するということを、脳の中で完結できる人もいれば、それが苦手な人もいます。 なので、一度脳の中にある考えを口に出してすべて吐き出してしまうことで、一度脳の中で考えていることすべてを出力します。 それをGPTの力を使って構造化、整理することによって、自分ではなんとなくモヤモヤと考えていたことを話しするだけで構造化された、整理整頓された考えとして、 自分の手の内に入れることができるというツールを開発しています。 思考だったりコーパイロットみたいな役割になっていると考えています。 なので、このツールを今作っているので、そのツールの名前をちょっと考えたいなと思っています。 このツールの名前として、これらの特徴をいろいろと整理したいなと考えています。 キーワードとしては、思考、コーパイロット、GPT、言語、考え、整理、AI。 うーん。 内容。


In [ ]:
# 要約処理

# チャットモデルのラッパーをインポート
from langchain.chat_models import ChatOpenAI
# チャットプロンプト用のテンプレートをインポート
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# チャットモデルのラッパーを初期化
chat = ChatOpenAI(
    model_name="gpt-4", # gpt-4が利用できなければコメントアウトしてください
    temperature=0
    )

# SystemMessage 用のテンプレートの作成
template="あなたは提示された文章の文脈や構造を理解し、理解しやすい形に構造化・整理することが得意です。"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# HumanMessage 用のテンプレートの作成
human_template="以下の文章の文脈を汲み取り、誤字脱字と推測される部分を適切に修正した上で、構造化した上で人間が理解しやすい形のMarkdown形式で出力してください。「{text}」"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Message のテンプレートを組合わせて会話の流れを決めます
messages_template = [
    system_message_prompt,
    human_message_prompt
]

# チャットプロンプト用のテンプレートを作成します
chat_prompt_template = ChatPromptTemplate.from_messages(messages_template)

# テンプレートに具体値を組み込んでチャットプロンプトを作成します
chat_prompt = chat_prompt_template.format_prompt(text=talk_script).to_messages()

# チャットの補完を作成
completion = chat(chat_prompt)
print("あなたの考えを整理してみました！")
print("")
print(completion.content)

あなたの考えを整理してみました！
# GPTを使った思考整理ツール

## 概要
- GPTを使って自分の考えや思考を整理するツールを開発中
- 脳の中の煩雑な情報を構造化・整理することで、深い理解や洞察を得られると考えている
- 考えを整理・構造化するのが得意な人もいれば、苦手な人もいるため、このツールが役立つと考えている

## ツールの仕組み
1. 脳の中にある考えを口に出してすべて吐き出す
2. GPTの力を使って構造化・整理する
3. モヤモヤと考えていたことが構造化された、整理整頓された考えとして手に入れることができる

## ツールの役割
- 思考の整理やコーパイロットのような役割を果たす

## ツール名の検討
- 特徴を整理してツール名を考えたい
- キーワード: 思考、コーパイロット、GPT、言語、考え、整理、AI
- 内容を考慮してツール名を決定する
